In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

from gangen.cdcgan import Generator, Discriminator

/home/takayuki/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
data_dir = '/home/takayuki/Desktop/sem6/DL/mini_proj_TRAFFIC/data/'
train_dir = os.path.join(data_dir, 'preprocessed', 'Train')

data_tf = transforms.Compose([
    transforms.RandomAffine(degrees=10, translate=(0.08, 0.08), scale=(0.9, 1.1), shear=5),
    transforms.Resize((32, 32)),
    
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
   
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 1.0))], p=0.3),

    transforms.ToTensor(),

    transforms.RandomErasing(p=0.2, scale=(0.01, 0.04), ratio=(0.5, 2.0), value=0), # value=0 for black box

    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])


full_dataset = datasets.ImageFolder(root = train_dir, transform=data_tf)

In [ ]:
# Hyperparameters
batch_size = 64
num_epochs = 50
beta1 = 0.5
beta2 = 0.999
noise_dim = 100
n_classes = 43
embedding_dim = 64
lambda_gp = 10

k_steps_D = 1
k_steps_G = 1 
log_interval = 5  # Log images every N epochs
checkpoint_interval = 10 # Save checkpoint every N epochs

label_smoothing = 0.0
learning_rate_g = 0.0002
learning_rate_d = 0.00025

In [5]:
data_loader = DataLoader(
    full_dataset, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    drop_last=True,
    pin_memory=True
)

targets = full_dataset.targets
class_counts = np.bincount(targets, minlength=n_classes)
class_counts = np.array(class_counts, dtype=np.float32)
    
print("Class counts:", class_counts)

class_weights = 1.0 / class_counts
class_weights = class_weights / np.sum(class_weights) * n_classes  # Normalize
class_weights = torch.FloatTensor(class_weights).to(device)

Class counts: [ 210. 2220. 2010. 1320. 2100. 2160.  780.  630.  420. 1110. 1200.  210.
 2250.  360.  330.  390.  510.  270. 1500.  600.  240.  540.  270. 1410.
  450.  780.  240.  689.  420. 1200.  390.  210. 2070.  300. 1980.  360.
  240.  240. 1860.  420. 1440. 1410. 1470.]


In [6]:
for inp, lab in data_loader:
    print("Input shape:", inp.shape)
    print("Label shape:", lab.shape)
    break

Input shape: torch.Size([64, 3, 32, 32])
Label shape: torch.Size([64])


In [7]:
G = Generator(noise_dim=noise_dim, n_classes=n_classes, embedding_dim=embedding_dim).to(device)
D = Discriminator(n_classes=n_classes, embedding_dim=embedding_dim).to(device)

# Set up optimizers
optimizer_G = optim.Adam(G.parameters(), lr=learning_rate_g, betas=(beta1, beta2))
optimizer_D = optim.Adam(D.parameters(), lr=learning_rate_d, betas=(beta1, beta2))

# Loss function 
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.MSELoss()

In [8]:
# Setup TensorBoard
log_dir = 'logs/gan_training_' + time.strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir)

In [9]:
def generate_samples(num_samples=16, fixed_noise=None, fixed_labels=None):
    G.eval()
    with torch.no_grad():
        if fixed_noise is None:
            fixed_noise = torch.randn(num_samples, noise_dim, device=device)
        if fixed_labels is None:
            # Generate samples across different classes
            fixed_labels = torch.arange(0, min(n_classes, num_samples), device=device)
            # Repeat labels if num_samples > n_classes
            fixed_labels = fixed_labels.repeat(num_samples // min(n_classes, num_samples) + 1)[:num_samples]
        
        fake_samples = G(fixed_noise, fixed_labels)
        
    # Convert to displayable format
    fake_samples = fake_samples.detach().cpu().numpy()
    # Move channel dimension to the end for plotting
    fake_samples = np.transpose(fake_samples, (0, 2, 3, 1))
    # Denormalize
    fake_samples = fake_samples * 0.5 + 0.5
    fake_samples = np.clip(fake_samples, -1, 1)
    
    return fake_samples, fixed_labels.cpu().numpy()

# Create a grid of images
def create_image_grid(images, labels, nrow=4):
    ncol = images.shape[0] // nrow
    fig, axes = plt.subplots(nrow, ncol, figsize=(12, 12))
    
    for i, ax in enumerate(axes.flat):
        if i < images.shape[0]:
            ax.imshow(images[i])
            ax.set_title(f"Class: {labels[i]}")
            ax.axis('off')
    
    plt.tight_layout()
    return fig

# Function to save checkpoint
def save_checkpoint(G, D, optimizer_G, optimizer_D, epoch, filename='checkpoint.pth'):
    torch.save({
        'epoch': epoch,
        'generator_state_dict': G.state_dict(),
        'discriminator_state_dict': D.state_dict(),
        'optimizer_G_state_dict': optimizer_G.state_dict(),
        'optimizer_D_state_dict': optimizer_D.state_dict(),
    }, filename)

# Create fixed noise for visualization
fixed_noise = torch.randn(16, noise_dim, device=device)
fixed_labels = torch.arange(0, min(n_classes, 16), device=device).repeat(16 // min(n_classes, 16) + 1)[:16]



In [10]:
d_loss_history = []
g_loss_history = []

d_real_acc_history = []
d_fake_acc_history = []

print(f"Starting training with k_steps_D={k_steps_D}, k_steps_G={k_steps_G}...")

for epoch in range(num_epochs):
    G.train()
    D.train()

    # Metrics accumulated over the epoch
    epoch_d_loss = 0.0
    epoch_g_loss = 0.0
    epoch_d_real_acc = 0.0 # Tracks accuracy on real images from the last D step in each batch cycle
    epoch_d_fake_acc = 0.0 # Tracks accuracy on fake images from the last D step in each batch cycle
    num_d_updates = 0
    num_g_updates = 0

    # Use tqdm for progress bar
    progress_bar = tqdm(enumerate(data_loader), total=len(data_loader), desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for i, (real_images, real_labels) in progress_bar:

        real_images = real_images.to(device)
        real_labels = real_labels.to(device)
        current_batch_size = real_images.size(0)

        # Create targets for loss calculation
        # Note: D uses smoothed real targets, G uses non-smoothed targets typically
        real_target_smoothed = torch.full((current_batch_size, 1), (1.0 - label_smoothing), device=device)
        real_target_unsmoothed = torch.ones(current_batch_size, 1, device=device) # For G loss
        fake_target = torch.zeros(current_batch_size, 1, device=device)

        # --- Store metrics for the current batch cycle ---
        current_d_loss = 0.0
        current_g_loss = 0.0
        current_d_real_acc = 0.0
        current_d_fake_acc = 0.0

        # ---------------------------------
        # Train Discriminator k_steps_D times
        # ---------------------------------
        for _ in range(k_steps_D):
            optimizer_D.zero_grad()

            # --- Loss on Real Images ---
            d_real_output = D(real_images, real_labels)
            if d_real_output.shape != real_target_smoothed.shape:
                 raise ValueError(f"Shape mismatch D real: D output {d_real_output.shape}, target {real_target_smoothed.shape}")
            d_real_loss = criterion(d_real_output, real_target_smoothed)

            # --- Loss on Fake Images ---
            z = torch.randn(current_batch_size, noise_dim, device=device)
            # Sample fake labels respecting class distribution
            fake_labels_dist = torch.multinomial(class_weights, current_batch_size, replacement=True)

            # Generate fake images - detach G's graph
            with torch.no_grad():
                fake_images = G(z, fake_labels_dist).detach()

            d_fake_output = D(fake_images, fake_labels_dist)
            if d_fake_output.shape != fake_target.shape:
                 raise ValueError(f"Shape mismatch D fake: D output {d_fake_output.shape}, target {fake_target.shape}")
            d_fake_loss = criterion(d_fake_output, fake_target)

            # --- Total D Loss and Update ---
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            optimizer_D.step()

            # --- Store metrics from this D step ---
            current_d_loss = d_loss.item() # Store the latest D loss for postfix
            epoch_d_loss += current_d_loss # Accumulate total D loss
            num_d_updates += 1

            # Calculate accuracy only for the *last* D update in this cycle for epoch avg.
            if _ == k_steps_D - 1:
                 current_d_real_acc = ((torch.sigmoid(d_real_output) > 0.5).float().mean()).item() # Use sigmoid for accuracy check
                 current_d_fake_acc = ((torch.sigmoid(d_fake_output) < 0.5).float().mean()).item() # Use sigmoid for accuracy check
                 epoch_d_real_acc += current_d_real_acc
                 epoch_d_fake_acc += current_d_fake_acc


        # ---------------------------------
        # Train Generator k_steps_G times
        # ---------------------------------
        # Only proceed if G needs updates (prevents unnecessary computation if k_steps_G=0)
        if k_steps_G > 0:
            # Prevent D gradients from affecting G updates
            # (May not be strictly necessary if D params aren't used in G loss path,
            # but good practice)
            for p in D.parameters():
                p.requires_grad = False

            for _ in range(k_steps_G):
                optimizer_G.zero_grad()

                # --- Generate New Fake Data ---
                z = torch.randn(current_batch_size, noise_dim, device=device)
                # Sample fake labels respecting class distribution
                fake_labels_dist_g = torch.multinomial(class_weights, current_batch_size, replacement=True)
                fake_images_g = G(z, fake_labels_dist_g)

                # --- Calculate G Loss (trying to fool D) ---
                g_output = D(fake_images_g, fake_labels_dist_g)

                # Use non-smoothed real targets for G's objective
                if g_output.shape != real_target_unsmoothed.shape:
                    raise ValueError(f"Shape mismatch G: D output {g_output.shape}, target {real_target_unsmoothed.shape}")

                # G wants D to predict fake images as real (output logits close to 1)
                g_loss = criterion(g_output, real_target_unsmoothed)

                # --- Backpropagation and Update G ---
                g_loss.backward()
                optimizer_G.step()

                # --- Store metrics from this G step ---
                current_g_loss = g_loss.item() # Store the latest G loss for postfix
                epoch_g_loss += current_g_loss # Accumulate total G loss
                num_g_updates += 1

            # Re-enable D gradients for the next D update cycle
            for p in D.parameters():
                p.requires_grad = True


        # --- Update Progress Bar ---
        # Display the losses/accuracies from the *last* D and G steps of this batch cycle
        progress_bar.set_postfix({
            'D Loss': f"{current_d_loss:.4f}",
            'G Loss': f"{current_g_loss:.4f}" if k_steps_G > 0 else "N/A",
            'D Real Acc': f"{current_d_real_acc:.3f}",
            'D Fake Acc': f"{current_d_fake_acc:.3f}"
        })
        progress_bar.update(1) # Increment progress bar

    progress_bar.close() # Close the tqdm bar for the epoch

    # --- End of Epoch Calculations and Logging ---
    # Calculate average losses and accuracies for the epoch
    avg_d_loss = epoch_d_loss / num_d_updates if num_d_updates > 0 else 0
    avg_g_loss = epoch_g_loss / num_g_updates if num_g_updates > 0 else 0
    # Average accuracies are over the number of batch cycles (len(data_loader))
    avg_d_real_acc = epoch_d_real_acc / len(data_loader) if len(data_loader) > 0 else 0
    avg_d_fake_acc = epoch_d_fake_acc / len(data_loader) if len(data_loader) > 0 else 0

    d_loss_history.append(avg_d_loss)
    g_loss_history.append(avg_g_loss)
    d_real_acc_history.append(avg_d_real_acc)
    d_fake_acc_history.append(avg_d_fake_acc)

    # Log metrics to TensorBoard
    writer.add_scalar('Loss/Discriminator_Avg', avg_d_loss, epoch)
    writer.add_scalar('Loss/Generator_Avg', avg_g_loss, epoch)
    writer.add_scalar('Accuracy/D_Real_Avg', avg_d_real_acc, epoch)
    writer.add_scalar('Accuracy/D_Fake_Avg', avg_d_fake_acc, epoch)

    print(f"Epoch [{epoch+1}/{num_epochs}] Summary - "
          f"Avg D Loss: {avg_d_loss:.4f}, Avg G Loss: {avg_g_loss:.4f}, "
          f"Avg D Real Acc: {avg_d_real_acc:.3f}, Avg D Fake Acc: {avg_d_fake_acc:.3f}\n")

    # --- Generate and Save Sample Images ---
    if (epoch + 1) % log_interval == 0 or epoch == 0 or epoch == num_epochs - 1:
        G.eval() # Set G to evaluation mode for generation
        fake_samples, sample_labels = generate_samples(num_samples=16, fixed_noise=fixed_noise, fixed_labels=fixed_labels)
        fig = create_image_grid(fake_samples, sample_labels, nrow=4) # Assuming nrow=4

        # Save figure to TensorBoard
        writer.add_figure(f'Generated Traffic Signs/Epoch {epoch+1}', fig, global_step=epoch)

        # Save figure to disk
        os.makedirs('generated_samples', exist_ok=True)
        fig.savefig(f'generated_samples/epoch_{epoch+1:04d}.png') # Use padding for sorting
        plt.close(fig) # Close the figure to free memory
        G.train() # Set G back to training mode

    # --- Save Checkpoint ---
    if (epoch + 1) % checkpoint_interval == 0 or epoch == num_epochs - 1:
        os.makedirs('checkpoints', exist_ok=True)
        
        # delete old checkpoints to save space
        checkpoints = sorted(os.listdir('checkpoints'))
        if len(checkpoints) > 5:
            for old_checkpoint in checkpoints[:-1]:
                os.remove(os.path.join('checkpoints', old_checkpoint))
                print(f"Deleted old checkpoint: {old_checkpoint}")
                
        save_checkpoint(G, D, optimizer_G, optimizer_D, epoch, f'checkpoints/cgan_gtsrb_ksteps_epoch_{epoch+1:04d}.pth')
        print(f"Checkpoint saved for epoch {epoch+1}.")

# --- End of Training ---
# Save final model explicitly
save_checkpoint(G, D, optimizer_G, optimizer_D, num_epochs, 'checkpoints/cgan_gtsrb_ksteps_final.pth')

print("Training complete!")
writer.close()

Starting training with k_steps_D=1, k_steps_G=1...


Epoch [1/50] Summary - Avg D Loss: 0.1810, Avg G Loss: 0.7527, Avg D Real Acc: 0.995, Avg D Fake Acc: 0.044



Epoch [2/50] Summary - Avg D Loss: 0.2799, Avg G Loss: 0.7248, Avg D Real Acc: 0.992, Avg D Fake Acc: 0.033



Epoch [3/50] Summary - Avg D Loss: 0.2389, Avg G Loss: 0.6729, Avg D Real Acc: 0.998, Avg D Fake Acc: 0.015



Epoch [4/50] Summary - Avg D Loss: 0.2401, Avg G Loss: 0.6756, Avg D Real Acc: 0.993, Avg D Fake Acc: 0.028



...

Epoch [9/50] Summary - Avg D Loss: 0.2786, Avg G Loss: 0.5582, Avg D Real Acc: 0.995, Avg D Fake Acc: 0.000



Epoch [10/50] Summary - Avg D Loss: 0.2758, Avg G Loss: 0.5569, Avg D Real Acc: 0.998, Avg D Fake Acc: 0.000

Checkpoint saved for epoch 10.


Epoch [11/50] Summary - Avg D Loss: 0.2748, Avg G Loss: 0.5622, Avg D Real Acc: 0.998, Avg D Fake Acc: 0.000



Epoch [12/50] Summary - Avg D Loss: 0.2757, Avg G Loss: 0.5640, Avg D Real Acc: 0.998, Avg D Fake Acc: 0.000



KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import os

# --- Assume Training Loop Finished ---
# Make sure these lists exist and are populated correctly:
# d_loss_history = [...] # Avg D loss per epoch
# g_loss_history = [...] # Avg G loss per epoch
# d_real_acc_history = [...] # Avg D real accuracy per epoch
# d_fake_acc_history = [...] # Avg D fake accuracy per epoch

print("\nPlotting loss and accuracy history...")

# --- Directory for Plots ---
plot_dir = 'plots'
os.makedirs(plot_dir, exist_ok=True)

# --- Plot Losses ---
if d_loss_history and g_loss_history:
    epochs = range(1, len(d_loss_history) + 1)
    plt.figure(figsize=(12, 6))
    plt.plot(epochs, g_loss_history, label='Generator Average Loss', color='deepskyblue')
    plt.plot(epochs, d_loss_history, label='Discriminator Average Loss', color='salmon')
    plt.title('Average Generator and Discriminator Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (BCEWithLogitsLoss)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # Save the loss plot
    loss_plot_filename = os.path.join(plot_dir, 'gan_loss_history.png')
    try:
        plt.savefig(loss_plot_filename)
        print(f"Loss graph saved to: {loss_plot_filename}")
    except Exception as e:
        print(f"Error saving loss graph: {e}")
    plt.show() # Optional: Show interactively
    plt.close() # Close the figure
else:
    print("Skipping loss plot (history lists missing or empty).")


# --- Plot Accuracies ---
# Make sure history lists for accuracy exist
if d_real_acc_history and d_fake_acc_history:    
    
    # Define epochs if not already defined (in case loss plot was skipped)
    if 'epochs' not in locals():
        epochs = range(1, len(d_real_acc_history) + 1)

    plt.figure(figsize=(12, 6))
    # Plot the ACTUAL accuracy histories
    plt.plot(epochs, d_real_acc_history, label='Discriminator Real Accuracy', color='forestgreen')
    plt.plot(epochs, d_fake_acc_history, label='Discriminator Fake Accuracy', color='darkorange')
    plt.title('Average Discriminator Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1.05) # Set Y-axis limits for accuracy (0 to 1)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    # Save the accuracy plot
    acc_plot_filename = os.path.join(plot_dir, 'gan_accuracy_history.png')
    try:
        plt.savefig(acc_plot_filename)
        print(f"Accuracy graph saved to: {acc_plot_filename}")
    except Exception as e:
        print(f"Error saving accuracy graph: {e}")
    plt.show() # Optional: Show interactively
    plt.close() # Close the figure
else:
    print("Skipping accuracy plot (history lists missing or empty).")

In [ ]:
# Save the final model explicitly
save_name = "goodrun_c2"
save_path = os.path.join('models', f'{save_name}.pth')
os.makedirs('models', exist_ok=True)

# save the model weights

torch.save({
    'generator_state_dict': G.state_dict(),
    'discriminator_state_dict': D.state_dict(),
    'optimizer_G_state_dict': optimizer_G.state_dict(),
    'optimizer_D_state_dict': optimizer_D.state_dict(),
}, save_path)
print(f"Final model saved to: {save_path}")

In [ ]:
# Generate final samples
fake_samples, sample_labels = generate_samples(num_samples=36)
fig = create_image_grid(fake_samples, sample_labels, nrow=6)
# fig.savefig('final_generated_samples.png')
plt.show()